In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
from sklearn import metrics
import tensorflow as tf
import os
import pandas as pd
import librosa
import glob

Read k fold and make patient wise LSTM models and do testing to get accuracy

In [2]:
DATA_PATH_kfold = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/cis-pd.training_data.k_fold/"

data_dir = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"
model_dir = "/home/sjoshi/codes/python/BeatPD/code/models/"
data_type = 'cis'

In [3]:
k_fold_file_list=glob.glob(DATA_PATH_kfold+"*.csv")

In [4]:
fileNamesList = [] # List of all file names (without full path)
for x in k_fold_file_list:
     split_results = x.split('/')
     fileNamesList.append(split_results[-1])

In [5]:
patientNamesList = []
for x in fileNamesList:
     split_results = x.split('_')
     patientNamesList.append(split_results[0])

In [6]:
def define_data_type(data_type, training_or_ancillary='training_data', data_real_subtype=None):
    """
    Setup file names
    
    Keyword arguments:
    - data_type = {cis , real}
    - training_or_ancillary: {'training_data', 'ancillary_data'} to switch between both
    - data_real_subtype: only provided for REAL-PD 

    If data_type is real, data_real_subtype will have to be declared as well 
    data_real_subtype={smartphone_accelerometer, smartwatch_accelerometer, smartwatch_gyroscope}
    """
    if data_type == "cis":
        path_train_labels = (
            data_dir
            + data_type
            + "-pd.data_labels/"
            + data_type.upper()
            + "-PD_"
            + ("Training_Data" if training_or_ancillary == 'training_data' else "Ancillary_Data")
            +"_IDs_Labels.csv"
        )
        path_train_data = data_dir + data_type + "-pd."+training_or_ancillary+"/"

    if data_type == "real":
        path_train_labels = (
            data_dir
            + data_type
            + "-pd.data_labels/"
            + data_type.upper()
            + "-PD_"
            + ("Training_Data" if training_or_ancillary == 'training_data' else "Ancillary_Data")
            +"_IDs_Labels.csv"
        )
        
        path_train_data = (
            data_dir + data_type + "-pd."+training_or_ancillary+"/" + data_real_subtype + "/"
        )

    # Display labels
    df_train_label = pd.read_csv(path_train_labels)
    
    # The time doesn't have the same name depending on the data_type
    x_axis_data_type = "t" if data_type == "real" else "Timestamp"

    cols_to_norm = ["x", "y", "z"] if data_type == "real" else ["X", "Y", "Z"]
    
    return path_train_data, df_train_label, x_axis_data_type, cols_to_norm

In [10]:
def get_files_kfold(fileNamesList,patientID):
    #patient_files_full_path = [i for i in k_fold_file_list if patientID in i] 
    patient_files = [i for i in fileNamesList if patientID in i]   
   
    kFoldsListForPatient = []
    for x in patient_files:
        y=x.replace('.csv', '')
        split_results = y.split('_')
        kFoldsListForPatient.append(split_results[-1])
    kFoldsListForPatient = list(set(kFoldsListForPatient)) # remove duplicates
    return kFoldsListForPatient


'''
def ThreeD(a, b, c): 
    lst1 = [[ ['1' for col in range(a)] for col in range(b)] for row in range(c)] 
    lst2= [[ ['2' for col in range(a)] for col in range(b)] for row in range(c)] 
    # Merging using "+" operator 
    lst = lst1+lst2 
    return lst
'''

'\ndef ThreeD(a, b, c): \n    lst1 = [[ [\'1\' for col in range(a)] for col in range(b)] for row in range(c)] \n    lst2= [[ [\'2\' for col in range(a)] for col in range(b)] for row in range(c)] \n    # Merging using "+" operator \n    lst = lst1+lst2 \n    return lst\n'

In [83]:
#for patientID in patientNamesList[0]:
patientID=patientNamesList[0] ## To debug

kFoldsListForPatient=get_files_kfold(fileNamesList,patientID)

kFoldsListForPatient=kFoldsListForPatient[0] ## To debug

for k in kFoldsListForPatient:
 
    train_file=DATA_PATH_kfold+patientID+'_train_kfold_'+k+'.csv'
    df_train_label = pd.read_csv(train_file)
    df_train_label = df_train_label.dropna(axis=1, how='all')
    df_train_label = df_train_label.dropna(axis=1, how='all')
    
    test_file=DATA_PATH_kfold+patientID+'_test_kfold_'+k+'.csv'
    df_test_label = pd.read_csv(test_file)
    df_test_label = df_test_label.dropna(axis=1, how='all')
    

    labelHeaders = df_train_label.columns.values[2:] # Select the columns for which patient can be trained
    
    path_train_data, notreq, x_axis_data_type, cols_to_norm = define_data_type(data_type, 
                                                      training_or_ancillary='training_data', 
                                                      data_real_subtype=None)
    for modelID in range(len(labelHeaders)):
        # Get all train data in correct form
        X_train=np.zeros((1, 1, 3))
        y_train=[]
        for idx in df_train_label.index:
            df_train_data = pd.read_csv(path_train_data + df_train_label["measurement_id"][idx] + ".csv")
            df_train_data_xyz=df_train_data[cols_to_norm]
            
            ## To do: Read files from measurement_id and use as X_train
            if idx==0:
                X_train[0,:,:]=df_train_data_xyz.to_numpy()
            else:
                X_train=np.concatenate((X_train,df_train_data_xyz.to_numpy()))
                
            ## y_train are labelHeaders's contents (are in float, might need change to int)
            y_train.append(df_train_label[labelHeaders].iloc[idx].values)
            
            #save model @ model_save_path=model_dir+data_type+'/'
        
            # Model ID are the number of models to be trained
    

ValueError: could not broadcast input array from shape (59852,3) into shape (1,3)

In [90]:
df_train_data_xyz.to_numpy().shape

(59852, 3)

In [ ]:

# printing result  
print(kFoldsListForPatient)
print ("l strings with given substring are : " + str(patient_files)) 



In [ ]:
# Useful Constants
# Adapted from : https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [ ]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train_all = load_X(X_train_signals_paths)
X_test_all = load_X(X_test_signals_paths)


# Keeping only acc #and gyro (remove gyro and total)
X_train=X_train_all[:,:,0:3]
X_test=X_test_all[:,:,0:3]


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Substract 1 to each output class for friendly 0-based indexing
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)


In [ ]:
X_train.shape

In [ ]:
#[Sonal]
def change_labels_to_two_class(y_train):
    '''
    Change labels to two class (activity=1, inactivity=0)
    
    LABELS = [
    "WALKING", 0
    "WALKING_UPSTAIRS", 1
    "WALKING_DOWNSTAIRS", 2
    "SITTING", 3
    "STANDING", 4
    "LAYING" 5
    
    '''
    y_train_new=np.where(y_train==0, 1, y_train) 
    y_train_new=np.where(y_train_new==1, 1, y_train_new)
    y_train_new=np.where(y_train_new==2, 1, y_train_new) 
    y_train_new=np.where(y_train_new==3, 0, y_train_new) # Replace sitting by inactivity
    y_train_new=np.where(y_train_new==4, 0, y_train_new)
    y_train_new=np.where(y_train_new==5, 0, y_train_new)
    print(np.unique(y_train_new))
    return y_train_new

y_train=change_labels_to_two_class(y_train)
y_test=change_labels_to_two_class(y_test)

In [ ]:
LABELS_NEW=['activity','inactivity']

In [ ]:
# Input Data

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep ---- Changed to 6!!![Sonal]



# LSTM Neural Network's internal structure

#n_hidden = 32 # Hidden layer num of features
n_hidden = 16 # Hidden layer num of features
n_classes = 2 # Total classes (should go up, or should go down) # [Sonal]


# Training

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

In [ ]:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input])
    # new shape: (n_steps*batch_size, n_input)

    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data.

    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index]

    return batch_s


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes
    # e.g.:
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [ ]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs,
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)

    # Evaluate network only at some steps for faster training:
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):

        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))

        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy],
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

In [ ]:
# (Inline plots: )
%matplotlib inline

font = {
    'family' : 'Bitstream Vera Sans',
    'weight' : 'bold',
    'size'   : 18
}
matplotlib.rc('font', **font)

width = 12
height = 12
plt.figure(figsize=(width, height))

indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

indep_test_axis = np.append(
    np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
    [training_iters]
)
plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training iteration')

plt.show()

In [ ]:
# Results

predictions = one_hot_predictions.argmax(1)

print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)

# Plot Results:
width = 10
height = 10
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix,
    interpolation='nearest',
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
plt.xticks(tick_marks, LABELS_NEW, rotation=90)
plt.yticks(tick_marks, LABELS_NEW)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Using pre-trained model on our data

In [ ]:
def define_data_type(data_type):
    # Setup file names

    '''
    data_type={cis , real}

    If data_type is real, data_real_subtype
    data_real_subtype={smartphone_accelerometer , smartwatch_accelerometer , smartwatch_gyroscope}
    '''
    if data_type=='cis':
        path_train_labels=data_dir+data_type+'-pd.data_labels/'+data_type.upper()+'-PD_Training_Data_IDs_Labels.csv'
        path_train_data=data_dir+data_type+'-pd.training_data/'

    if data_type=='real':
        path_train_labels=data_dir+data_type+'-pd.data_labels/'+data_type.upper()+'-PD_Training_Data_IDs_Labels.csv'
        path_train_data=data_dir+data_type+'-pd.training_data/'+data_real_subtype+'/'

    # Display labels
    df_train_label=pd.read_csv(path_train_labels)
    return path_train_data, df_train_label

In [ ]:
# Data paths

data_dir='/home/sjoshi/codes/python/BeatPD/data/BeatPD/'

In [ ]:
data_type='cis' 
#data_real_subtype='smartphone_accelerometer' 
#data_real_subtype='smartwatch_accelerometer'
#data_real_subtype='smartwatch_gyroscope'


In [ ]:
path_train_data, df_train_label = define_data_type(data_type=data_type)

# Array concatenating the training data so we can train the ubm with everything
nd_train_data = np.empty((0,3), float)
list_train_data = []
for idx in df_train_label.index:
    df_train_data=pd.read_csv(path_train_data+df_train_label["measurement_id"][idx]+'.csv')
    x = df_train_data.iloc[:,-3:]
    list_train_data.append(x.to_numpy())
    nd_train_data = np.append(nd_train_data, x.to_numpy(), axis=0)

In [ ]:
# Make a windowing fcn 
# Source:https://krbnite.github.io/Memory-Efficient-Windowing-of-Time-Series-Data-in-Python-Memory-Strides-in-NumPy/
def make_windows(
  arr,
  win_size,
  step_size,):
  """
  arr: any 2D array whose columns are distinct variables and 
    rows are data records at some timestamp t
  win_size: size of data window (given in data points)
  step_size: size of window step (given in data point)
  
  Note that step_size is related to window overlap (overlap = win_size - step_size), in 
  case you think in overlaps.
  """
  w_list = list()
  n_records = arr.shape[0]
  remainder = (n_records - win_size) % step_size 
  num_windows = 1 + int((n_records - win_size - remainder) / step_size)
  for k in range(num_windows):
    w_list.append(arr[k*step_size:win_size-1+k*step_size+1])
  return np.array(w_list)

In [ ]:
frame_length=128
frame_step=64
path_train_data, df_train_label = define_data_type(data_type=data_type)
idx=1
df_train_data=pd.read_csv(path_train_data+df_train_label["measurement_id"][idx]+'.csv')


In [ ]:
signal=tf.convert_to_tensor(df_train_data[['X','Y','Z']].values)

In [ ]:
framed_signal=tf.signal.frame(signal,frame_length,frame_step,pad_end=True,pad_value=0,axis=-1,name=None)
unframed_signal=tf.signal.overlap_and_add(framed_signal,frame_step,name=None)

In [ ]:
tf.version()

In [ ]:
win_size=128
step_size=64


for idx in df_train_label.index:
 

    path_train_data_1, df_train_label_1 = define_data_type(data_type=data_type)
    data_real_subtype='smartwatch_gyroscope'
    path_train_data_2, df_train_label_2 = define_data_type(data_type=data_type)


    df_train_data_accel=pd.read_csv(path_train_data+df_train_label_1["measurement_id"][idx]+'.csv')
    df_train_data_gyros=pd.read_csv(path_train_data+df_train_label_2["measurement_id"][idx]+'.csv')

    df_train_data_accel = df_train_data_accel.drop(['t'], axis=1)
    df_train_data_gyros = df_train_data.drop(['t'], axis=1)

    result = pd.concat([df_train_data_accel,df_train_data_gyros],axis=1)

    win_data = make_windows(result.values, win_size, step_size)
    
    if idx == 0:
        train_data=win_data
    else:
        train_data=np.concatenate((train_data, win_data))


In [ ]:
X_train.shape

In [ ]:
df_train_data_only = df_train_data.drop(['t'], axis=1)
df_train_data_only.values.shape

In [ ]:
# How to store mask?
# 